In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
rc('font', family = 'Malgun Gothic')
rc('axes', unicode_minus = False)
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [3]:
driver =  webdriver.Chrome()
driver.get('https://www.musinsa.com/categories/item/005?d_cat_cd=005&brand=&list_kind=small&sort=pop_category&sub_sort=&page=3&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=')

ul_element = driver.find_element(By.CSS_SELECTOR, '#searchList') 
li_elements = ul_element.find_elements(By.CSS_SELECTOR, 'li')  

data_no_list = []
for li in li_elements:
    data_no = li.get_attribute('data-no') 
    if data_no != None:
        data_no_list.append(data_no)

driver.quit()

In [4]:
data_no_list

['3856694',
 '3835869',
 '4045751',
 '3339924',
 '4060063',
 '3348616',
 '2898906',
 '3315960',
 '1645143',
 '2871174',
 '3982339',
 '2378989',
 '2433538',
 '3813327',
 '3270661',
 '3027317',
 '3075621',
 '3018128',
 '3197839',
 '3882160',
 '2574791',
 '3772755',
 '3745950',
 '3255891',
 '2363570',
 '3775911',
 '3348606',
 '3367823',
 '3033947',
 '1438345',
 '3119471',
 '2440524',
 '3128355',
 '3974581',
 '1439527',
 '2645639',
 '3242240',
 '3976330',
 '3804587',
 '3568230',
 '2613418',
 '3286833',
 '3408438',
 '2833023',
 '3388877',
 '3405424',
 '3403236',
 '2101924',
 '2816108',
 '1032784',
 '3004188',
 '3940857',
 '3197911',
 '3331579',
 '3931876',
 '3428245',
 '3327675',
 '1129750',
 '3932871',
 '3961388',
 '3135860',
 '2091749',
 '3969783',
 '3914838',
 '2306631',
 '3923778',
 '3948168',
 '3413034',
 '3882166',
 '3317858',
 '3250621',
 '2611696',
 '3352448',
 '3467922',
 '3853098',
 '2599641',
 '3491717',
 '3027325',
 '1878236',
 '4014646',
 '4030307',
 '2231916',
 '3936752',
 '23

In [ ]:
# tag_lists = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-evaluation--type3 > ul")
# tag_lists = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-evaluation--type3 > div") 인것도 있음...

In [ ]:
# 리뷰가 없는 페이지 오류 떳을때 넘어가겠끔하는 확인용 코드

btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']

data_no = '4061819' # 리뷰가 없는 아이템
driver = webdriver.Chrome()
driver.get('https://www.musinsa.com/app/goods/'+data_no)
driver.fullscreen_window()

category, sex, price = get_product_info(driver, data_no)
print(f"Scraping reviews for product {data_no}...")
all_df = scrape_reviews(driver, btn_list, category, sex, price)

In [ ]:
# 리뷰있는 아이템 / 리뷰없는 아이템 / 리뷰있는 아이템으로 잘 진행되는지 확인해보기

final_df = pd.DataFrame()
data_no_list = ['3835869','4061819','4045751']


for data_no in data_no_list:
        btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']
        
        driver = webdriver.Chrome()
        driver.get('https://www.musinsa.com/app/goods/'+data_no)
        driver.fullscreen_window()

        category, sex, price = get_product_info(driver, data_no)
        
        if category and sex and price:
            print(f"Scraping reviews for product {data_no}...")
            all_df = scrape_reviews(driver, btn_list, category, sex, price)
            final_df = pd.concat([final_df,all_df]) 
        else:
            print(f"Skipping product {data_no} due to missing information.")

       

In [ ]:
# 최종 입력 코드
# 중요 : 웹크롤링 시작전에 data_no_list는 원하는 페이지 아이템으로 설정하기

final_df = pd.DataFrame()

for data_no in data_no_list:
        btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']
        
        driver = webdriver.Chrome()
        driver.get('https://www.musinsa.com/app/goods/'+data_no)
        driver.fullscreen_window()

        category, sex, price = get_product_info(driver, data_no)
        
        if category and sex and price:
            print(f"Scraping reviews for product {data_no}...")
            all_df = scrape_reviews(driver, btn_list, category, sex, price)
            final_df = pd.concat([final_df,all_df]) 
        else:
            print(f"Skipping product {data_no} due to missing information.")

In [ ]:
# [최종 통합 코드]
# 중요 : 웹크롤링 시작전에 data_no_list는 원하는 페이지 아이템으로 설정하기

# 상품 정보를 가져오는 함수
def get_product_info(driver, data_no):
    try:
        driver.get('https://www.musinsa.com/app/goods/' + data_no)
        driver.fullscreen_window()

        category = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-2.bIfCpH > div.product-detail__sc-up77yl-0.KuCLD > a:nth-child(3)').text
        try:
            sex = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-achptn-0.bHXxTQ > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.TehCn > span:nth-child(3)').text
        except NoSuchElementException:    
            sex = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-achptn-0.bHXxTQ > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.TehCn > span').text
        price = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-w5wkld-0.hgCYZm > div.product-detail__sc-1p1ulhg-0.jEclp > ul > li:nth-child(1) > div.product-detail__sc-1p1ulhg-6.fKNtEN > span').text
        price = price.replace(',', '').replace('원', '').strip()
        return category, sex, price
    
    except NoSuchElementException as e:
        print(f"An error occurred while getting product info: {e}")
        return None, None, None

# 3가지 후기 버튼 클릭 함수
def click_button(driver, btn):
    try:
        review_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, btn)))
        review_btn.click()
        time.sleep(3)
        return True
    except Exception as e:
        print(f"버튼 클릭 중 오류: {e}")
        return False

# 최대 페이지 수 추출 함수
def get_max_page(driver):
    try:
        texts = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.box_page_msg')))
        text = texts.text
        page_str = text.split(' ')[0]  # 텍스트에서 첫 번째 숫자를 추출
        page_max = int(page_str)
        return page_max
    except Exception as e:
        print(f"페이지 정보 수집 중 오류: {e}")
        return 0

# 리뷰 데이터 수집 함수
def collect_reviews_from_page(driver, btn, data, category, sex, price):
    try:
        review_lists = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#reviewListFragment')))
        review_list = review_lists.find_elements(By.CSS_SELECTOR, 'div.review-list')

        if not review_list:
            print(f"No reviews found for button {btn}.")
            return

        for review in review_list:
            try:
                name = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__name").text
                date = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__date").text
                product = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > a").text
                option = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > p > span").text
                score_str = review.find_element(By.CSS_SELECTOR, "div.review-list__rating-wrap > span > span > span").get_attribute('style')
                score = int(re.search(r'width:\s*(\d+)%;', score_str).group(1)) / 20
                review_text = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-contents__text").text
                help_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(1) > label > span").text

                if btn == '#estimate_style':
                    style_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(2) > label > span").text
                    review_type = 'style'
                elif btn == '#estimate_photo':
                    style_cnt = None
                    review_type = 'product'
                else:
                    style_cnt = None
                    review_type = 'general'

                tag_lists = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-evaluation--type3 > ul")
                tag_list = tag_lists.find_elements(By.CSS_SELECTOR, 'li.review-evaluation--type3__item')
                tags = ', '.join(tag.text.replace('\n', '-') for tag in tag_list)

                data.append({
                    "data_no": len(data) + 1,
                    "name": name,
                    "date": date,
                    "product": product,
                    "detail": option,
                    "score": score,
                    "review": review_text,
                    "help_cnt": help_cnt,
                    "style_cnt": style_cnt,
                    "type": review_type,
                    "category": category,
                    "sex": sex,
                    "price": price,
                    "tags": tags
                })
            except Exception as e:
                print(f"리뷰 데이터 추출 오류: {e}")
                continue
    except Exception as e:
        print(f"리뷰 페이지 수집 중 오류: {e}")

# 리뷰 스크랩 함수
def scrape_reviews(driver, btn_list, category, sex, price):
    all_df = pd.DataFrame()

    for btn in btn_list:
        if not click_button(driver, btn):
            continue
        
        page_max = get_max_page(driver)
        if page_max == 0:
            continue
        
        data = []
        for num in range(page_max):
            try:
                print('current page:', num + 1)
                n = num % 5
                page_btn_lists = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div')))
                page_btn = page_btn_lists.find_elements(By.CSS_SELECTOR, 'a')
                page_btn[n + 2].click()  # [2],[3] .. [6] => 1페이지, 2페이지 ... 5페이지 클릭
                time.sleep(3)

                collect_reviews_from_page(driver, btn, data, category, sex, price)

                if num % 5 == 4:
                    next_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next')))
                    next_btn.click()
                    time.sleep(3)
            except Exception as e:
                print(f"페이지 {num + 1} 처리 중 오류: {e}")
                continue
        
        df = pd.DataFrame(data)
        all_df = pd.concat([all_df, df], ignore_index=True)

    return all_df

# 메인 코드
final_df = pd.DataFrame()

for data_no in data_no_list:
    driver = webdriver.Chrome()
    btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']

    category, sex, price = get_product_info(driver, data_no)
    
    if category and sex and price:
        print(f"Scraping reviews for product {data_no}...")
        all_df = scrape_reviews(driver, btn_list, category, sex, price)
        final_df = pd.concat([final_df, all_df], ignore_index=True)
    else:
        print(f"Skipping product {data_no} due to missing information.")
    
    driver.quit()


In [ ]:
#원본코드_by 김유현님

all_df = pd.DataFrame()
btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']
data_no_list = ['3835869','4061819','4045751']

for data_no in data_no_list:
    try:
        driver =  webdriver.Chrome()
        driver.get('https://www.musinsa.com/app/goods/'+data_no)
        driver.fullscreen_window()
        
        category = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-2.bIfCpH > div.product-detail__sc-up77yl-0.KuCLD > a:nth-child(3)').text
        try:
            sex = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-achptn-0.bHXxTQ > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.TehCn > span:nth-child(3)').text
        except:    
            sex = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-achptn-0.bHXxTQ > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.TehCn > span').text
        
        price = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-w5wkld-0.hgCYZm > div.product-detail__sc-1p1ulhg-0.jEclp > ul > li:nth-child(1) > div.product-detail__sc-1p1ulhg-6.fKNtEN > span').text
        price = price.replace(',','').replace('원','').replace(' ','')

        for btn in btn_list:
            review_btn = driver.find_element(By.CSS_SELECTOR, btn)
            review_btn.click()
            time.sleep(3)

            texts = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.box_page_msg')
            text = texts.text
            page_str = text[0:text.find('페')]
            page_str = page_str.replace(' ','')
            page_cnt = int(page_str)
            print(f"Scraping reviews for product {data_no}...")
            print(f'max page {btn} === {page_cnt}')

            data = []

            for num in range(page_cnt):

                print('current page:',num+1)
                n = num % 5 
                page_btn_lists = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div') 
                page_btn = page_btn_lists.find_elements(By.CSS_SELECTOR, 'a') 
                page_btn[n+2].click()
                time.sleep(3)

                review_lists = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment') 
                review_list = review_lists.find_elements(By.CSS_SELECTOR, 'div.review-list')  

                for review in review_list:
                    name = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__name").text
                    date = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__date").text
                    product = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > a").text
                    option = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > p > span").text
                    score_str = review.find_element(By.CSS_SELECTOR, "div.review-list__rating-wrap > span > span > span").get_attribute('style')
                    score = int(re.search(r'width:\s*(\d+)%;', score_str).group(1))/20
                    review_text = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-contents__text").text
                    help_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(1) > label > span").text

                    if btn == '#estimate_style': 
                        style_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(2) > label > span").text
                        review_type = 'style'
                    elif btn == '#estimate_photo':
                        style_cnt =  None
                        review_type = 'product'
                    else:
                        style_cnt =  None
                        review_type = 'general'
                    
                    
                    tag_lists = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-evaluation--type3 > ul")
                    tag_list = tag_lists.find_elements(By.CSS_SELECTOR, 'li.review-evaluation--type3__item')
                  
                    tags = ''
                    
                    for tag in tag_list:
                        tags = tags + tag.text.replace('\n','-') +', '
                    tags = tags[:len(tags)-2]

                    data.append({
                        "data_no" : data_no,
                        "name": name,
                        "date": date,
                        "product": product,
                        "detail": option,
                        "score": score,
                        "review": review_text,
                        "help_cnt": help_cnt,
                        "style_cnt": style_cnt,
                        "type": review_type,
                        "category": category,
                        "sex": sex,
                        "price": price,
                        "tag": tags
                    })

                if num % 5 == 4: #num은 0,1,2,3,4로 진행된다. 
                    next_btn = driver.find_element(By.CSS_SELECTOR,'#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next')
                    next_btn.click()
                    time.sleep(3)

            df = pd.DataFrame(data)
            all_df = pd.concat([all_df,df])

        driver.quit()
    except NoSuchElementException as e:
        print(f"An error occurred: {e}")
        driver.quit()
        continue

In [ ]:
final_df

In [ ]:
final_df.to_csv('../data/review_3page.csv', encoding='utf-8')